<a href="https://colab.research.google.com/github/Ojas-Mehta/RNNResearchSentimentAnalysis/blob/master/SentimentAnalysisCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import torch.nn as nn
import pickle
import json
import pandas as pd
%matplotlib inline

In [0]:
# !pip3 install git+https://github.com/PetrochukM/PyTorch-NLP.git
# from torchnlp.word_to_vector import GloVe
# vectors = GloVe(name  = 'twitter.27B',dim=50)

In [0]:
# pickle.dump(a, open(f'/content/drive/My Drive/sentiment/pretrained.pkl', 'wb'))
# words = pickle.load(open(f'/content/drive/My Drive/sentiment/pretrained.pkl', 'rb'))
# print(words)

In [0]:

# #Generate synthetic data
# #train_synth.csv, test_synth.csv
# import numpy as np
# import csv
# dataset = []
# dataset_size = 2000
# for i in range(dataset_size):
#     a = np.floor(100*np.random.rand(10)).astype(int)
#     if(((80 <= a) & (a < 89)).sum()>=((90 <= a) & (a < 100)).sum()):
#       label = 1
#     else:
#       label = 0
#     dataset.append([a.tolist(),label])
# with open('/content/drive/My Drive/sentiment/data/val_small.csv', 'w') as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerows(dataset)
# csvFile.close()


In [0]:
#read data
import pandas as pd
train_data = pd.read_csv('/content/drive/My Drive/sentiment/data/train_small.csv', header = None)
val_data = pd.read_csv('/content/drive/My Drive/sentiment/data/val_small.csv', header = None)

test_data = pd.read_csv('/content/drive/My Drive/sentiment/data/test_small.csv', header = None)
train_X_list = train_data[0].tolist()
train_y = train_data[1].tolist()
val_X_list = val_data[0].tolist()
val_y = val_data[1].tolist()
test_X_list = test_data[0].tolist()
test_y = test_data[1].tolist()


In [0]:
#Preprocesssing: Numeric from string
torch.manual_seed(2109)
def stringToList(string):
    string = string[1:len(string)-1]
    try:
        if len(string) != 0: 
            tempList = string.split(", ")
            newList = list(map(lambda x: int(x), tempList))
        else:
            newList = []
    except:
        newList = [-9999]
    return(newList)
train_X_list = [stringToList(x) for x in train_X_list]
train_X = torch.tensor(train_X_list, dtype=torch.long)
val_X_list = [stringToList(x) for x in val_X_list]
val_X = torch.tensor(val_X_list, dtype=torch.long)
test_X_list = [stringToList(x) for x in test_X_list]
test_X = torch.tensor(test_X_list, dtype=torch.long)

In [0]:
# Define hyperparameters
input_dim = 10
output_dim = 2
n_epochs = 150
batch_size = 100
lr = 0.001
hidden_dimension = 8
patience_param = 4

In [0]:
torch.manual_seed(2109)
#Get embeddings
embeddings = nn.Embedding(100, input_dim)
train_input = embeddings(train_X)
train_output = torch.tensor(train_y, dtype=torch.long)

val_input = embeddings(val_X)
val_output = torch.tensor(val_y, dtype=torch.long)
test_input = embeddings(test_X)
result = []
# pickle.dump({'train':train_input, 'trainy':train_y, 'validation':val_input, 'valy':val_y, 'test':test_input},open(f'/content/drive/My Drive/sentiment/Results/InputEmbeddings.pkl', 'wb'))


In [14]:
torch.manual_seed(2109)
#Code used for TRAINING
def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

def evaluate(tempmodel, dataX, dataY, criterion):
  
    epoch_loss = 0
    epoch_acc = 0
    correct_pred = 0
    tempmodel.eval()
    with torch.no_grad():
        for i in range(0, dataX.shape[0], batch_size):
            batchX, batchy = dataX[i:i+batch_size], dataY[i:i+batch_size]
            batchX = batchX.to(device)
            batchy = batchy.to(device)
            output, hidden = tempmodel(batchX)          
            loss = criterion(hidden, batchy)

            epoch_loss += loss.item()*batch_size
            _, predicted_labels = torch.max(hidden, 1)
            correct_pred += (predicted_labels==batchy).sum().item()
        accuracy = correct_pred/ len(dataX)*100
        epoch_loss /= len(dataX)   
    return epoch_loss, accuracy

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")
    
    
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)
#         print(out.shape)
#         print(hidden.shape)

        hidden = hidden.transpose(0,1).contiguous().view(batch_size, -1)
        hidden = self.fc(hidden)
#         print(out.shape, hidden.shape)
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        hidden = hidden.to(device)
        return hidden
      
# Instantiate the model with hyperparameters
model = Model(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dimension, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)



# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

printLoss, printAccuracy = evaluate(model, train_input, train_output, criterion)
print("Training Loss and Accuracy: {:.4f}, {:.4f}".format(printLoss, printAccuracy))
printLoss, printAccuracy = evaluate(model, val_input, val_output, criterion)
print("Validation Loss and Accuracy: {:.4f}, {:.4f}".format(printLoss, printAccuracy))
oldAcc = printAccuracy
k = 0
for epoch in range(1, n_epochs + 1):
    model.train()
    for i in range(0, train_input.shape[0], batch_size):
        optimizer.zero_grad()
        batch_X, batch_y = train_input[i:i+batch_size], train_output[i:i+batch_size]
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        output, hidden = model(batch_X)
        loss = criterion(hidden, batch_y)
        loss.backward(retain_graph=True) # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly
    
    if 1:
        
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        printLoss, printAccuracy = evaluate(model, train_input, train_output, criterion)
        print("Training Loss and Accuracy: {:.4f}, {:.4f}".format(printLoss, printAccuracy))
        printLoss, printAccuracy = evaluate(model, val_input, val_output, criterion)
        print("Validation Loss and Accuracy: {:.4f}, {:.4f}".format(printLoss, printAccuracy))
        if(oldAcc>printAccuracy):
          lr/=2
          k+=1
        else:
          k=0
        if(k>=patience_param):
          break
        oldAcc = printAccuracy
result = []
with open('/content/drive/My Drive/sentiment/Results/Result8ss.json', 'w') as f:
  result.append({epoch: str(model.state_dict())})
  json.dump(result, f)
# torch.save(model.state_dict(), '/content/drive/My Drive/sentiment/Model/myModelsmall.pth')
torch.save(model.state_dict(), '/content/drive/My Drive/sentiment/Model/myModelsmall8ss.pt')

GPU is available
Training Loss and Accuracy: 0.7566, 40.2667
Validation Loss and Accuracy: 0.7626, 38.8500
Epoch: 1/150............. Training Loss and Accuracy: 0.6551, 62.8667
Validation Loss and Accuracy: 0.6512, 63.9500
Epoch: 2/150............. Training Loss and Accuracy: 0.6434, 63.1000
Validation Loss and Accuracy: 0.6379, 64.9000
Epoch: 3/150............. Training Loss and Accuracy: 0.6260, 65.1600
Validation Loss and Accuracy: 0.6188, 65.9500
Epoch: 4/150............. Training Loss and Accuracy: 0.6163, 65.8800
Validation Loss and Accuracy: 0.6085, 66.3000
Epoch: 5/150............. Training Loss and Accuracy: 0.6071, 66.6133
Validation Loss and Accuracy: 0.5988, 66.8500
Epoch: 6/150............. Training Loss and Accuracy: 0.5926, 67.7467
Validation Loss and Accuracy: 0.5840, 67.7500
Epoch: 7/150............. Training Loss and Accuracy: 0.5700, 69.3800
Validation Loss and Accuracy: 0.5620, 70.5000
Epoch: 8/150............. Training Loss and Accuracy: 0.5293, 72.8800
Validation 

In [0]:
#########Inference
input_dim = 10
output_dim = 2
n_epochs = 50
batch_size = 100
lr=0.001

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")
    
    
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)

        hidden = hidden.transpose(0,1).contiguous().view(batch_size, -1)
        hidden = self.fc(hidden)
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        hidden = hidden.to(device)
        return hidden

# Instantiate the model with hyperparameters
model = Model(input_size=input_dim, output_size=output_dim, hidden_dim=10, n_layers=1)
model.load_state_dict(torch.load('/content/drive/My Drive/sentiment/Model/myModelsmall10.pt'))
# model.eval()
# model = model.to(device)
# batch_X, batch_y = train_input[0:1], train_output[0:1]
# batch_X = batch_X.to(device)
# batch_y = batch_y.to(device)
# output, hidden = model(batch_X)

GPU is available


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
import json
result = []
with open('/content/drive/My Drive/sentiment/Results/SecondResultSmall.json', 'w') as f:
          json.dump(result, f)

In [0]:
#save the model
# torch.save(model.state_dict(), '/content/drive/My Drive/sentiment/Model/')
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")
    
    
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)
#         print(out.shape)
#         print(hidden.shape)

        hidden = hidden.transpose(0,1).contiguous().view(batch_size, -1)
        hidden = self.fc(hidden)
#         print(out.shape, hidden.shape)
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        hidden = hidden.to(device)
        return hidden
#Load the model

model = Model(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dimension, n_layers=1)
# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model.load_state_dict(torch.load('/content/drive/My Drive/sentiment/Model/myModelsmall10.pt'))
# model.eval()

GPU is available


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
#########Read JSON File
import json
with open('/content/drive/My Drive/sentiment/Results/FirstResult.json', 'r') as f:
  result = json.loads(f.read())
print(len(result))

147


In [10]:
########Load Model
torch.manual_seed(2109)
#Code
def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

def evaluate(tempmodel, dataX, dataY, criterion):
  
    epoch_loss = 0
    epoch_acc = 0
    correct_pred = 0
    tempmodel.eval()
    with torch.no_grad():
        for i in range(0, dataX.shape[0], batch_size):
            batchX, batchy = dataX[i:i+batch_size], dataY[i:i+batch_size]
            batchX = batchX.to(device)
            batchy = batchy.to(device)
            output, hidden = tempmodel(batchX)          
            loss = criterion(hidden, batchy)

            epoch_loss += loss.item()*batch_size
            _, predicted_labels = torch.max(hidden, 1)
            correct_pred += (predicted_labels==batchy).sum().item()
        accuracy = correct_pred/ len(dataX)*100
        epoch_loss /= len(dataX)   
    return epoch_loss, accuracy

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")
    
    
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)
#         print(out.shape)
#         print(hidden.shape)

        hidden = hidden.transpose(0,1).contiguous().view(batch_size, -1)
        hidden = self.fc(hidden)
#         print(out.shape, hidden.shape)
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        hidden = hidden.to(device)
        return hidden
      
# Instantiate the model with hyperparameters
model = Model(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dimension, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.load_state_dict(torch.load('/content/drive/My Drive/sentiment/Model/myModelsmall8.pt'))
model.to(device)


GPU is available


Model(
  (rnn): RNN(10, 8, batch_first=True)
  (fc): Linear(in_features=8, out_features=2, bias=True)
)

In [11]:
###Get parameters of the loaded model
ordir = model.state_dict()
print(ordir)
InputParam = ordir['rnn.weight_ih_l0'].cpu()
InputBias = ordir['rnn.bias_ih_l0'].cpu()
HiddenParam = ordir['rnn.weight_hh_l0'].cpu()
HiddenBias = ordir['rnn.bias_hh_l0'].cpu()

OrderedDict([('rnn.weight_ih_l0', tensor([[ 0.5346,  1.0179,  0.0586, -0.1215, -0.7051, -0.5493,  0.2434,  0.1060,
         -0.4051, -0.5529],
        [-0.3803, -0.2369,  0.7409,  0.3470, -0.1311, -0.0642,  0.1315, -0.9807,
         -0.2692, -0.2011],
        [-0.7092,  0.3890,  0.0813, -0.1533,  0.1883,  0.3681,  0.1788,  0.3680,
         -0.2529,  0.1657],
        [-0.3872,  0.1718,  0.3645,  0.3122,  0.2762,  0.5664, -0.2306,  0.6595,
         -0.5076,  0.2364],
        [ 0.5229,  0.6768,  0.0530, -0.2626, -0.2817,  0.2029,  0.2200, -0.3312,
         -0.2490, -0.6428],
        [-0.7145, -0.0238,  0.6449,  0.4349, -0.1739, -0.3180, -0.0571,  0.0758,
          0.4326,  0.3535],
        [ 0.1014, -0.1125, -0.0243,  0.3305,  0.1069, -0.0590, -0.0108,  0.0933,
         -0.1592, -0.1502],
        [-0.6407,  0.0577, -0.4438,  0.3906,  0.5159, -0.2522,  0.1429,  0.2993,
          0.4600, -0.1553]], device='cuda:0')), ('rnn.weight_hh_l0', tensor([[ 0.1411,  0.1243, -0.1780,  0.1757, -0.1130,

In [13]:
##############################Product of parameter and input word vector
# train_input[0].to(device)
# # InputParam[0].to(device)
# InputParam[0]*train_input[0]
# train_input.to(device)
for i in range(100):
    inputword = embeddings(torch.tensor(i, dtype=torch.long))
    print(torch.norm(torch.mm(InputParam,inputword.view(10,1))+InputBias.view(8,1)))#default fr
#     print(torch.sum(torch.mm(InputParam,inputword.view(10,1))+InputBias.view(8,1)))

tensor(2.8004, grad_fn=<NormBackward0>)
tensor(3.7301, grad_fn=<NormBackward0>)
tensor(3.5162, grad_fn=<NormBackward0>)
tensor(3.6228, grad_fn=<NormBackward0>)
tensor(3.6680, grad_fn=<NormBackward0>)
tensor(2.3133, grad_fn=<NormBackward0>)
tensor(3.9492, grad_fn=<NormBackward0>)
tensor(2.2190, grad_fn=<NormBackward0>)
tensor(3.0481, grad_fn=<NormBackward0>)
tensor(2.0381, grad_fn=<NormBackward0>)
tensor(2.3953, grad_fn=<NormBackward0>)
tensor(2.5231, grad_fn=<NormBackward0>)
tensor(2.0880, grad_fn=<NormBackward0>)
tensor(3.8336, grad_fn=<NormBackward0>)
tensor(5.0081, grad_fn=<NormBackward0>)
tensor(4.0756, grad_fn=<NormBackward0>)
tensor(3.5097, grad_fn=<NormBackward0>)
tensor(3.3934, grad_fn=<NormBackward0>)
tensor(3.7256, grad_fn=<NormBackward0>)
tensor(2.6356, grad_fn=<NormBackward0>)
tensor(3.9251, grad_fn=<NormBackward0>)
tensor(3.3370, grad_fn=<NormBackward0>)
tensor(4.4730, grad_fn=<NormBackward0>)
tensor(2.2138, grad_fn=<NormBackward0>)
tensor(3.0495, grad_fn=<NormBackward0>)
